## messy things to remember:

1. starting around 12:00am 8/1, I changed the bandcamp album scraper to:
    - only scrape a maximum of 10 albums
    - do not scrape from urls that contain 'productions' or 'records' to going down the record label holes of 100s of albums
    
2. as of ~11am on 8/1, I completely redid the sql tables

In [59]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import tqdm
import numpy as np

In [60]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'music_db'
username = 'shayneufeld'

In [61]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://shayneufeld@localhost/music_db


In [4]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


# create album supporter and album info tables fromd bandcamp album scraper

In [45]:
info_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_info'
supporter_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_supporters'
files_info = os.listdir(info_folder)
files_supporter = os.listdir(supporter_folder)
files_stored_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/supp_files_stored_0801.csv',index_col=0)
files_stored = files_stored_df.files_stored.values

In [46]:
info_master = pd.DataFrame()
#files_stored = []
for info in tqdm.tqdm(files_info):
    if info not in files_stored:
        temp_info = pd.read_csv(os.path.join(info_folder,info),index_col = 0)
        info_master = info_master.append(temp_info)


100%|██████████| 5478/5478 [00:56<00:00, 96.13it/s] 


In [47]:
info_master.shape

(2354, 468)

In [48]:
info_master.index = np.arange(info_master.shape[0])

In [49]:
col_dict = {}
df = pd.DataFrame(columns=['Bundle','Vinyl','CD','Tape','Digital','Unclassified'])
df['album_name'] = info_master.album_name
df['artist_name'] = info_master.artist_name
df['location'] = info_master.location
df['tags'] = info_master.tags
df['year'] = info_master.year

In [50]:
info_master.loc[info_master.year.isnull(),'year'] = 1900

In [51]:
for i in tqdm.tqdm(np.arange(info_master.shape[0])):
    curr = info_master.iloc[i]
    a = curr[~pd.isnull(curr)]
    cols = a.index
    cols = cols.drop(['album_name','artist_name','location','tags','year'])
    
    for col in cols:
        if (('Bundle' in col) or ('Discography' in col) or ('bundle' in col)):
            #print('Bundle: %s' % col)
            #col_dict.update({col:'Bundle'})
            df.loc[i,'Bundle'] = info_master.loc[i,col]
                   
        elif (('Vinyl' in col) or ('12"' in col) or ('vinyl' in col) or ('LP' in col)):
            #print('Vinyl: %s' % col)
            #col_dict.update({col:'Vinyl'})
            df.loc[i,'Vinyl'] = info_master.loc[i,col]
                   
        elif (('Compact' in col) or ('CD' in col)):
            #print('CD: %s' % col)
            #col_dict.update({col:'CD'})
            df.loc[i,'CD'] = info_master.loc[i,col]
                   
        elif(('Cassette' in col) or ('Tape' in col) or ('CASSETTE' in col)):
            #print('Tape: %s' % col)
            #col_dict.update({col:'Tape'})
            df.loc[i,'Tape'] = info_master.loc[i,col]
                   
        elif(('Digital' in col) or ('Download' in col)):
            #print('Digital: %s' % col)
            #col_dict.update({col:'Digital'})
            df.loc[i,'Digital'] = info_master.loc[i,col]
                   
        else:
            #print('Unclassified: %s' % col)
            #col_dict.update({col:'Unclassified'})
            df.loc[i,'Unclassified'] = info_master.loc[i,col]
                   

100%|██████████| 2354/2354 [00:05<00:00, 450.62it/s]


In [52]:
df.shape

(2354, 11)

In [53]:
info_master.shape

(2354, 468)

In [38]:
df.head(50)

,Bundle,Vinyl,CD,Tape,Digital,Unclassified,album_name,artist_name,location,tags,year
0,NaN,NaN,NaN,NaN,$1,NaN,"""Condos""-LiveatTheBellHouse",Welcome to Night Vale,New York,['podcasts'],2014.0
1,(,NaN,NaN,NaN,70.4,NaN,"""Emerson""UnreleasedDemo",Wintergatan,Gothenburg,['pop'],2017.0
2,$60.30,NaN,NaN,NaN,$13,NaN,"""ILLESTRUSSIA""",PLAYA x TIGHT EYEZ x MMP,Ukraine,"['300', 'hip-hop/rap', 'instrumental hip-hop',...",2016.0
3,NaN,NaN,NaN,NaN,$1,NaN,"""TheDebate""-LiveatRoulette",Welcome to Night Vale,New York,"['podcasts', 'night vale']",2014.0
4,NaN,NaN,NaN,NaN,$5,NaN,"""TheInvestigators""Live",Welcome to Night Vale,New York,"['podcasts', 'cecil baldwin', 'comedy', 'night...",2016.0
5,NaN,NaN,NaN,NaN,$1,NaN,"""TheLibrarian""-LiveatSkirball",Welcome to Night Vale,New York,"['night vale', 'podcasts', 'welcome to night v...",2015.0
6,NaN,NaN,NaN,NaN,$7,NaN,"""Thisiswhatyoulearned.""",People Like You,Boston,"['alternative', 'broken world media', 'drum & ...",2014.0
7,NaN,NaN,NaN,NaN,NaN,NaN,#BASEDSHARK,Mayhem,Hamilton,['electronic'],2011.0
8,NaN,NaN,NaN,NaN,$10,$10,"#extrasinamovie|1.20.16|Raleigh,NC|TheRitz",Papadosio,Asheville,"['rock', 'alternative', 'electronic', 'improv'...",2016.0
9,NaN,NaN,NaN,NaN,$10,$10,"#extrasinamovie|1.21.16|Nashville,TN",Papadosio,Asheville,"['rock', 'alternative', 'electronic', 'improv'...",2016.0


In [54]:
#insert into db
df.to_sql('album_info',engine,if_exists='append')

In [55]:
supporter_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_supporters'
files_supp = os.listdir(supporter_folder)
supp_files_stored = []

# Add supporters files to DB

In [57]:
for supp in tqdm.tqdm(files_supporter):
    STORED = supp in files_stored

    if not STORED:
        temp_supp = pd.read_csv(os.path.join(supporter_folder,supp),index_col=0)
        temp_supp.to_sql('album_supporters',engine,if_exists='append')
        supp_files_stored.append(supp)

100%|██████████| 5478/5478 [00:34<00:00, 161.09it/s]


In [58]:
len(supp_files_stored)

2355

In [42]:
#change supp_files_stored to pandas so I can save it
files_stored_df = pd.DataFrame(data={'files_stored':supp_files_stored})
files_stored_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/supp_files_stored_0801.csv')

# Create spotify table

In [55]:
pwd

'/Users/shayneufeld/GitHub/pricecamp/notebooks'

In [256]:
spotify_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/spotify_df.csv',index_col=0)
spotify_df.to_sql('spotify_table', engine, if_exists='replace')

# Create bandcamp table

In [8]:
cd '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'

/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed


In [9]:
folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed'
files = os.listdir(folder)

for file in files:
    curr_df = pd.read_csv(file,index_col=0)
    if not len(curr_df) == 0:
        if curr_df.columns[0] == 'Unnamed: 0.1':
            curr_df = curr_df.drop('Unnamed: 0.1',axis=1)
        curr_df.to_sql('bandcamp_table',engine,if_exists='append')

# Create sales table

In [10]:
sales_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
sales_df.to_sql('sales_table', engine, if_exists='replace')

In [11]:
sales_df.shape

(74073, 22)

# Query from database

In [1]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM spotify_table WHERE match='True';
"""
sp_data_matches = pd.read_sql_query(sql_query,con)

sp_data_matches.shape[0]

NameError: name 'psycopg2' is not defined

In [262]:
sql_query = """
SELECT * FROM bandcamp_table;
"""
bandcamp_table = pd.read_sql_query(sql_query,con)

In [263]:
bandcamp_table.shape[0]

582344